In [116]:
import pandas as pd
import requests
from dotenv import load_dotenv
import os

In [117]:
%pip install psycopg2-binary

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [118]:

# Load the API key from .env file   
load_dotenv()
API_KEY = os.getenv("API_KEY")

# Fetch time series data for a given symbol.
def fetch_data(symbol):
    url = f"https://api.twelvedata.com/time_series?symbol={symbol}&interval=1min&apikey={API_KEY}"
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()  
    else:
        print(f"Failed to fetch data for {symbol}. Status code: {response.status_code}")
        return None

# List of symbols to fetch data for
symbols = ["AAPL", "EUR/USD", "BTC/USD"]

# Fetch and print data for each symbol
for symbol in symbols:
    data = fetch_data(symbol)
    if data:
        print(f"\n✅ Data for {symbol}:")
        print(data)
    else:
        print(f"\n❌ No data returned for {symbol}")




✅ Data for AAPL:
{'meta': {'symbol': 'AAPL', 'interval': '1min', 'currency': 'USD', 'exchange_timezone': 'America/New_York', 'exchange': 'NASDAQ', 'mic_code': 'XNGS', 'type': 'Common Stock'}, 'values': [{'datetime': '2025-07-21 15:59:00', 'open': '212.35001', 'high': '212.56000', 'low': '212.28000', 'close': '212.53999', 'volume': '890116'}, {'datetime': '2025-07-21 15:58:00', 'open': '212.25999', 'high': '212.35001', 'low': '212.21', 'close': '212.35001', 'volume': '222906'}, {'datetime': '2025-07-21 15:57:00', 'open': '212.28500', 'high': '212.34500', 'low': '212.23', 'close': '212.26500', 'volume': '174803'}, {'datetime': '2025-07-21 15:56:00', 'open': '212.20000', 'high': '212.32001', 'low': '212.14000', 'close': '212.28500', 'volume': '184871'}, {'datetime': '2025-07-21 15:55:00', 'open': '212.20000', 'high': '212.25', 'low': '212.079300', 'close': '212.19859', 'volume': '188359'}, {'datetime': '2025-07-21 15:54:00', 'open': '212.32001', 'high': '212.34000', 'low': '212.070007', 

In [119]:
data

{'meta': {'symbol': 'BTC/USD',
  'interval': '1min',
  'currency_base': 'Bitcoin',
  'currency_quote': 'US Dollar',
  'exchange': 'Coinbase Pro',
  'type': 'Digital Currency'},
 'values': [{'datetime': '2025-07-22 07:39:00',
   'open': '118326.75',
   'high': '118348.53',
   'low': '118272.13',
   'close': '118272.61'},
  {'datetime': '2025-07-22 07:38:00',
   'open': '118328.52',
   'high': '118333.78',
   'low': '118284.98',
   'close': '118326.7'},
  {'datetime': '2025-07-22 07:37:00',
   'open': '118417.56',
   'high': '118419.64',
   'low': '118328.49',
   'close': '118328.52'},
  {'datetime': '2025-07-22 07:36:00',
   'open': '118324.2',
   'high': '118440.71',
   'low': '118324.2',
   'close': '118410.26'},
  {'datetime': '2025-07-22 07:35:00',
   'open': '118273.07',
   'high': '118324.2',
   'low': '118273.07',
   'close': '118324.2'},
  {'datetime': '2025-07-22 07:34:00',
   'open': '118286.27',
   'high': '118310.25',
   'low': '118264.34',
   'close': '118267.45'},
  {'date

In [120]:
def transform_data(data, symbol=None):
    time_series = data['values']
    df = pd.DataFrame(time_series)

    # Add a new column for the symbol 
    if symbol:
        df['symbol'] = symbol

    # Convert 'datetime' to datetime object
    df['datetime'] = pd.to_datetime(df['datetime'])

    # Check if 'volume' column exists, if not, set it to NaN or omit it
    if 'volume' in df.columns:
        df['volume'] = df['volume'].astype(float)
    else:
        df['volume'] = pd.NA  # or use 'None' or 0 

    # Convert other columns to correct data types
    df = df.astype({
        'open': 'float',
        'high': 'float',
        'low': 'float',
        'close': 'float'
    })
    
    return df

In [121]:
# Process and combine data for all symbols
def process_symbols(symbols):
    all_data = []  # List to store transformed data for all symbols
    
    for symbol in symbols:
        # Fetch and transform the data for the current symbol
        data = fetch_data(symbol)
        if data:
            transformed_data = transform_data(data, symbol)
            all_data.append(transformed_data)
    
    # Combine all transformed data into a single DataFrame
    combined_df = pd.concat(all_data, ignore_index=True)
    return combined_df

# Process and combine data for all symbols
df_all_symbols = process_symbols(symbols)
print(df_all_symbols)

              datetime          open          high          low         close  \
0  2025-07-21 15:59:00     212.35001     212.56000     212.2800     212.53999   
1  2025-07-21 15:58:00     212.25999     212.35001     212.2100     212.35001   
2  2025-07-21 15:57:00     212.28500     212.34500     212.2300     212.26500   
3  2025-07-21 15:56:00     212.20000     212.32001     212.1400     212.28500   
4  2025-07-21 15:55:00     212.20000     212.25000     212.0793     212.19859   
..                 ...           ...           ...          ...           ...   
85 2025-07-22 07:14:00  118298.83000  118341.75000  118278.8300  118337.45000   
86 2025-07-22 07:13:00  118245.89000  118298.87000  118245.8900  118298.82000   
87 2025-07-22 07:12:00  118254.48000  118269.19000  118191.6800  118245.91000   
88 2025-07-22 07:11:00  118245.44000  118266.47000  118219.3400  118254.47000   
89 2025-07-22 07:10:00  118158.04000  118259.60000  118139.2500  118245.55000   

      volume   symbol  
0  

C:\Users\David Ibanga\AppData\Local\Temp\ipykernel_39464\3617618726.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(all_data, ignore_index=True)


In [122]:
# Save the DataFrame to CSV
df_all_symbols.to_csv('symbol_data.csv', index=False)


In [123]:
%pip install psycopg2-binary

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



## Data Loading

In [124]:
# Save to PostgreSQL
import psycopg2
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv
from urllib.parse import quote_plus

# PostgreSQL credentials
load_dotenv()
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER') or 'postgres'  # fallback to 'postgres' if not set
DB_PASSWORD = os.getenv('DB_PASSWORD')

# URL-encode the password to handle special characters like '@'
DB_PASSWORD_ENC = quote_plus(DB_PASSWORD)


# Create connection to PostgreSQL using SQLAlchemy engine
db_url = f'postgresql://{DB_USER}:{DB_PASSWORD_ENC}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
engine = create_engine(db_url)

# Save DataFrame to PostgreSQL database
df_all_symbols.to_sql('symbol_data_table', engine, if_exists='replace', index=False)

print("Data has been saved to both CSV and PostgreSQL.")






Data has been saved to both CSV and PostgreSQL.


# https://twelvedata.com/account/api-playground